

Author: Karshi Hasanov \
Date: May 3, 2023 \
Last Modified: May 9, 2023


# Getting Started With Python’s property()

Python’s **property()** is the Pythonic way to avoid formal getter and setter methods in your code. This function allows you to turn class attributes into properties or managed attributes. Since **property()** is a built-in function, you can use it without importing anything. Additionally, **property()** was implemented in C to ensure optimal performance.

```{note}
It’s common to refer to **property()** as a built-in function. However, the **property** is a class designed to work as a function rather than as a regular class. That’s why most Python developers call it a function. That’s also the reason why property() doesn’t follow the Python convention for naming classes.

This tutorial follows the common practices of calling property() a function rather than a class. However, in some sections, you’ll see it called a class to facilitate the explanation.
```

With property(), you can attach getter and setter methods to given class attributes. This way, you can handle the internal implementation for that attribute without exposing getter and setter methods in your API. You can also specify a way to handle attribute deletion and provide an appropriate docstring for your properties.

Here’s the full signature of property():
```python
property(fget=None, fset=None, fdel=None, doc=None)
```

| Argument | Description |
|----------|-------------|
| fget     | Function that returns the value of the managed attribute |
| fset     | Function that allows you to set the value of the managed attribute |
| fdel     | Function to define how the managed attribute handles deletion |
| doc      | String representing the property’s docstring |

## Creating Attributes With property()
You can create a property by calling property() with an appropriate set of arguments and assigning its return value to a class attribute. All the arguments to property() are optional. However, you typically provide at least a **setter function**.

The following example shows how to create a Circle class with a handy property to manage its radius:

```python
# circle.py
class Circle:
    def __init__(self, radius):
        self._radius = radius

    def _get_radius(self):
        print("Get radius")
        return self._radius

    def _set_radius(self, value):
        print("Set radius")
        self._radius = value

    def _del_radius(self):
        print("Delete radius")
        del self._radius

    radius = property(
        fget=_get_radius,
        fset=_set_radius,
        fdel=_del_radius,
        doc="The radius property."
    )
```



If the functionality of your getter method is limited to just returning the current value of the managed attribute, then using a **lambda** function can be a handy approach.

```python
radius = property(lambda self: self._radius)
```

Properties are **class attributes** that manage **instance attributes**. You can think of a property as a collection of methods bundled together. If you inspect .radius carefully, then you can find the raw methods you provided as the fget, fset, and fdel arguments.

Properties are also overriding descriptors. If you use dir() to check the internal members of a given property, then you’ll find **.\__set\__()** and **.\__get\__()** in the list. These methods provide a default implementation of the descriptor protocol.

The default implementation of **.\__set\__()**, for example, runs when you don’t provide a custom setter method. In this case, you get an **AttributeError** because there’s no way to set the underlying property.


## Using property() as a Decorator

Python’s **property()** can also work as a decorator, so you can use the **@property** syntax to create your properties quickly:
 

In [16]:
# circle.py
class Circle:
    def __init__(self, radius):
        self._radius = radius
        
    @property
    def radius(self):
        """The radius property."""
        print("Get radius")
        return self._radius
    
    @radius.setter
    def radius(self, value):
        print("Set radius")
        self._radius = value
        
    @radius.deleter
    def radius(self):
        print("Delete radius")
        del self._radius

```{note}
You don’t need to use a pair of parentheses for calling **.radius()** as a method. Instead, you can access **.radius** as you would access a regular attribute, which is the primary use of properties. They allow you to treat methods as attributes, and they take care of calling the underlying set of methods automatically.
```

In [17]:
circle = Circle(42.0)
circle.radius

Get radius


42.0

In [18]:
del circle.radius

Delete radius


## Providing Read-Only Attributes
Probably the most elementary use case of **property()** is to provide **read-only** attributes in your classes. Say you need an immutable Point class that doesn’t allow the user to mutate the original value of its coordinates, x and y. To achieve this goal, you can create Point like in the following example:

In [19]:
class Point:
    def __init__(self, x, y):
        self._x = x
        self._y = y

    @property
    def x(self):
        return self._x

    @property
    def y(self):
        return self._y

In [20]:
point = Point(1,2)
point.x

1

Now try to change the value:

In [21]:
point.x = 3

AttributeError: can't set attribute 'x'

Not only you can not change the values of the attributes **x** and **y**, but also you can not delete them:

In [22]:
del point.x

AttributeError: can't delete attribute 'x'

The reason for this is that if you do not implement the "**setter**" and "**deleter*" methods, by default
when you try to modify or delete the attributes it will rise the **AttributeError**.

You can take this implementation of the Point class a little bit further and provide explicit setter methods that raise a custom exception with more elaborate and specific messages:

In [23]:
# Lets define a custom Error Exception.
# Instead of the default "AttrubuteError", we want our class respond with some message.
class WriteCoordinateError(Exception):
    pass

class Point:
    def __init__(self, x, y):
        self._x = x
        self._y = y

    @property
    def x(self):
        return self._x

    @x.setter
    def x(self, value):
        raise WriteCoordinateError("x coordinate is read-only")

    @property
    def y(self):
        return self._y

    @y.setter
    def y(self, value):
        raise WriteCoordinateError("y coordinate is read-only")

## Providing Write-Only Attributes
You can also create **write-only** attributes by tweaking how you implement the getter method of your properties. For example, you can make your getter method raise an exception every time a user accesses the underlying attribute value.

Here’s an example of handling passwords with a write-only property:

In [25]:
# users.py

import hashlib
import os

class User:
    def __init__(self, username, password):
        self.username = username
        self.password = password

    @property
    def password(self):
        raise AttributeError("Password is write-only")

    @password.setter
    def password(self, plaintext):
        salt = os.urandom(32)
        self._hashed_password = hashlib.pbkdf2_hmac(
            "sha256", plaintext.encode("utf-8"), salt, 100_000
        )

In [26]:
peter = User('pcompton','his new password')
peter.password

AttributeError: Password is write-only

In [27]:
peter._hashed_password

b"P;\xe5[\xe0'a4\xe7\x82\xd8\x88\x15\xe0\xb2\x82f\x93e\x04X.\xc8u\xec\xe2G\xee\xd5/\x9d\x12"

## Putting Python’s property() Into Action
So far, you’ve learned how to use Python’s **property()** built-in function to create managed attributes in your classes. You used **property()** as a function and as a decorator and learned about the differences between these two approaches. You also learned how to create **read-only**, **read-write**, and **write-only** attributes.

In the following sections, you’ll code a few examples that will help you get a better practical understanding of common use cases of **property()**.

### Validating Input Values
One of the most common use cases of **property()** is building managed attributes that validate the input data before storing or even accepting it as a secure input. Data validation is a common requirement in code that takes input from users or other information sources that you consider untrusted.

Python’s **property()** provides a quick and reliable tool for dealing with input data validation. For example, thinking back to the Point example, you may require the values of .x and .y to be valid numbers. Since your users are free to enter any type of data, you need to make sure that your point only accepts numbers.

Here’s an implementation of Point that manages this requirement:

In [28]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    @property
    def x(self):
        return self._x

    @x.setter
    def x(self, value):
        try:
            self._x = float(value)
            print("Validated!")
        except ValueError:
            raise ValueError('"x" must be a number') from None

    @property
    def y(self):
        return self._y

    @y.setter
    def y(self, value):
        try:
            self._y = float(value)
            print("Validated!")
        except ValueError:
            raise ValueError('"y" must be a number') from None

The setter methods of **.x** and **.y** use **try … except** blocks that validate input data using the Python [EAFP](<https://docs.python.org/3/glossary.html#term-eafp>) style. If the call to **float()** succeeds, then the input data is valid, and you get Validated! on your screen. If **float()** raises a ValueError, then the user gets a ValueError with a more specific message.

```{note}
In the example above, you use the syntax **raise … from None** to hide internal details related to the context in which you’re raising the exception. From the end user’s viewpoint, these details can be confusing and make your class look unpolished.

Check out the section on the raise statement in the documentation for more information about this topic.
```
It’s important to note that assigning the **.x** and **.y** properties directly in **.\__init\__()** ensures that the validation also occurs during object initialization. Not doing so is a common mistake when using **property()** for data validation.

Here’s how your Point class works now:

In [29]:
point = Point(100,200)

Validated!
Validated!


In [30]:
point.x = 150

Validated!


In [31]:
point.y = 160.0

Validated!


In [32]:
point.x = "Five"

ValueError: "x" must be a number

The above code has uncovers a fundamental weakness of **property()** : The repetitive code that follows specific patterns. This repetition breaks the **DRY (Don’t Repeat Yourself)** principle, so you would want to refactor this code to avoid it. To do so, you can abstract out the repetitive logic using a descriptor:

In [33]:
class Coordinate:
    def __set_name__(self, owner, name):
        self._name = name

    def __get__(self, instance, owner):
        return instance.__dict__[self._name]

    def __set__(self, instance, value):
        try:
            instance.__dict__[self._name] = float(value)
            print("Validated!")
        except ValueError:
            raise ValueError(f'"{self._name}" must be a number') from None

class Point:
    x = Coordinate()
    y = Coordinate()

    def __init__(self, x, y):
        self.x = x
        self.y = y

Now your code is a bit shorter. You managed to remove repetitive code by defining Coordinate as a descriptor that manages your data validation in a single place. The code works just like your earlier implementation. Go ahead and give it a try!

In general, if you find yourself copying and pasting property definitions all around your code or if you spot repetitive code like in the example above, then you should consider using a proper descriptor.

## Providing Computed Attributes
If you need an attribute that builds its value dynamically whenever you access it, then property() is the way to go. These kinds of attributes are commonly known as **computed attributes**.
They’re handy when you need them to look like [eager](<https://en.wikipedia.org/wiki/Evaluation_strategy#Eager_evaluation>) attributes, but you want them to be [lazy](<https://en.wikipedia.org/wiki/Lazy_evaluation>).

Here’s an example of how to use property() to create a computed attribute .area in a Rectangle class:
```python
class Rectangle:
    def __init__(self, width, height):
        self.width = width
        self.height = height

    @property
    def area(self):
        return self.width * self.height
```

In this example, the Rectangle initializer takes width and height as arguments and stores them in regular instance attributes. The read-only property .area computes and returns the area of the current rectangle every time you access it.

Another common use case of properties is to provide an auto-formatted value for a given attribute:

```python
class Product:
    def __init__(self, name, price):
        self._name = name
        self._price = float(price)

    @property
    def price(self):
        return f"${self._price:,.2f}"
```

In this example, **.price** is a property that formats and returns the price of a particular product. To provide a currency-like format, you use an **f-string** with appropriate formatting options.

```{note}
This example uses floating-point numbers to represent currencies, which is bad practice. Instead, you should use [decimal.Decimal](<https://docs.python.org/3/library/decimal.html#decimal.Decimal>) from the standard library.
```
As a final example of computed attributes, say you have a **Point** class that uses .x and .y as Cartesian coordinates. You want to provide polar coordinates for your point so that you can use them in a few computations. The polar coordinate system represents each point using the distance to the origin and the angle with the horizontal coordinate axis.

Here’s a Cartesian coordinates Point class that also provides computed polar coordinates:


In [34]:
import math

class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    @property
    def distance(self):
        return round(math.dist((0, 0), (self.x, self.y)))

    @property
    def angle(self):
        return round(math.degrees(math.atan(self.y / self.x)), 1)

    def as_cartesian(self):
        return self.x, self.y

    def as_polar(self):
        return self.distance, self.angle

When it comes to providing computed or lazy attributes, property() is a pretty handy tool. However, if you’re creating an attribute that you use frequently, then computing it every time can be costly and wasteful. A good strategy is to cache them once the computation is done.

## Caching Computed Attributes
... Need some work here. I don't understand how the how the **cached_property** and **cache** from the **functools** module works.

## Logging Attribute Access and Mutation

Sometimes you need to keep track of what your code does and how your programs flow. A way to do that in Python is to use logging. This module provides all the functionality you would require for [logging](<https://realpython.com/python-logging/>) your code. It’ll allow you to constantly watch the code and generate useful information about how it works.

If you ever need to keep track of how and when you access and mutate a given attribute, then you can take advantage of **property()** for that, too:

In [35]:
import logging

logging.basicConfig(
    format="%(asctime)s: %(message)s",
    level=logging.INFO,
    datefmt="%H:%M:%S"
)

class Circle:
    def __init__(self, radius):
        self._msg = '"radius" was %s. Current value: %s'
        self.radius = radius

    @property
    def radius(self):
        """The radius property."""
        logging.info(self._msg % ("accessed", str(self._radius)))
        return self._radius

    @radius.setter
    def radius(self, value):
        try:
            self._radius = float(value)
            logging.info(self._msg % ("mutated", str(self._radius)))
        except ValueError:
            logging.info('validation error while mutating "radius"')

Here, you first import **logging** and define a basic configuration. Then you implement **Circle** with a managed attribute **.radius**. The getter method generates log information every time you access **.radius** in your code. The setter method logs each mutation that you perform on **.radius**. It also logs those situations in which you get an error because of bad input data.

Here’s how you can use **Circle** in your code:

In [36]:
circle = Circle(42.0)

16:02:24: "radius" was mutated. Current value: 42.0


In [37]:
circle.radius = 100

16:02:46: "radius" was mutated. Current value: 100.0


In [38]:
circle.radius = "value"

16:03:08: validation error while mutating "radius"


Logging useful data from attribute access and mutation can help you debug your code. Logging can also help you identify sources of problematic data input, analyze the performance of your code, spot usage patterns, and more.